In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import load_dataset
import json

# Load AG News dataset from Hugging Face
dataset = load_dataset("ag_news", split="train")

# Extract a subset of news headlines (first 100 for now)
df = pd.DataFrame(dataset[:100])
df = df.rename(columns={'text': 'headline'})

# Placeholder: Manually add satirical versions (for now, we use the same headline)
df["satirical_headline"] = df["headline"]

# Save as JSONL for OpenAI fine-tuning
jsonl_data = df[["headline", "satirical_headline"]].to_dict(orient="records")
with open("satirical_headlines.jsonl", "w") as f:
    for entry in jsonl_data:
        f.write(json.dumps(entry) + "\n")

print("Dataset saved as satirical_headlines.jsonl")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset saved as satirical_headlines.jsonl


In [4]:
!pip install openai


In [38]:
import openai
import os

# Set the API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Create an OpenAI client using the API key
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [39]:


# Test API call
try:
    client = openai.OpenAI()  # Create an OpenAI client
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI assistant."},
            {"role": "user", "content": "Hello, can you confirm that my OpenAI API key is working?"}
        ]
    )

    print("API Key is working! Response from GPT-4:")
    print(response.choices[0].message.content)

except Exception as e:
    print("Error:", e)


API Key is working! Response from GPT-4:
As an AI model developed by OpenAI, I don't have the access to personal data about individuals unless it has been shared with me during our conversation. I am designed to respect user privacy and confidentiality. Therefore, I'm unable to check or confirm your OpenAI API key status. You would need to test this on the OpenAI platform or through your implementation of the API.


In [11]:
!pip install ace_tools

In [14]:
import openai
import os
import json
import pandas as pd

# Load the dataset from the previously saved JSONL file
with open("satirical_headlines.jsonl", "r") as f:
    lines = f.readlines()

# Update to process 100 headlines instead of 10
num_samples = 100  # Increase sample size

# Select a larger subset of headlines
sample_headlines = [json.loads(line)["headline"] for line in lines[:num_samples]]

# Function to generate satirical headlines using GPT-4
def generate_satirical_headlines(headlines):
    satirical_headlines = []
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Ensure API key is set

    for headline in headlines:
        prompt = f"Rewrite the following news headline in a satirical, absurd, or humorous style, similar to The Onion:\n\nOriginal: {headline}\nSatirical:"

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an AI that rewrites news headlines in a satirical, humorous style similar to The Onion."},
                    {"role": "user", "content": prompt}
                ]
            )
            satirical_headline = response.choices[0].message.content.strip()
        except Exception as e:
            satirical_headline = f"Error generating satire: {str(e)}"

        satirical_headlines.append(satirical_headline)

    return satirical_headlines

# Generate satirical versions for 100 headlines
satirical_versions = generate_satirical_headlines(sample_headlines)

# Create DataFrame for comparison
satirical_data = pd.DataFrame({"Original Headline": sample_headlines, "Satirical Headline": satirical_versions})

# Save the updated dataset for fine-tuning
updated_jsonl_data = [{"headline": orig, "satirical_headline": sat} for orig, sat in zip(sample_headlines, satirical_versions)]
with open("satirical_headlines_updated.jsonl", "w") as f:
    for entry in updated_jsonl_data:
        f.write(json.dumps(entry) + "\n")

# Display the transformed headlines
from IPython.display import display
display(satirical_data)

print("Updated dataset saved as satirical_headlines_updated.jsonl")



,Original Headline,Satirical Headline
0,Wall St. Bears Claw Back Into the Black (Reute...,Wall Street's Extinct Bear Species Miraculousl...
1,Carlyle Looks Toward Commercial Aerospace (Reu...,"Carlyle Group Hedges Bets on Space, Declares A..."
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,(Oilspill News) - Stocks Bracing for Epic Wall...
3,Iraq Halts Oil Exports from Main Southern Pipe...,"Iraq Takes Pause from Oil Exports, Citing Thre..."
4,"Oil prices soar to all-time record, posing new...",US Economy Nervously Sweating as Oil Prices Pu...
...,...,...
95,Gene Blocker Turns Monkeys Into Workaholics - ...,Monkeys Hosting Corporate Seminars After Scien...
96,Dolphins Too Have Born Socialites (Reuters) Re...,"""Dolphins Discover Instagram, Immediately Floo..."
97,"What's in a Name? Well, Matt Is Sexier Than Pa...","""Groundbreaking Study Concludes Matts Worldwid..."
98,UK Scientists Allowed to Clone Human Embryos (...,"UK Scientists Secure 'Buy One, Get One Free' D..."


Updated dataset saved as satirical_headlines_updated.jsonl


In [15]:
import json

# Reload the dataset from the previous JSONL file
with open("satirical_headlines_updated.jsonl", "r") as f:
    lines = f.readlines()

# Extract original and satirical headlines
sample_headlines = [json.loads(line)["headline"] for line in lines]
satirical_versions = [json.loads(line)["satirical_headline"] for line in lines]

# Convert dataset into OpenAI fine-tuning format
fine_tuning_data = []

for orig, sat in zip(sample_headlines, satirical_versions):
    entry = {
        "messages": [
            {"role": "system", "content": "You are an AI that writes satirical headlines."},
            {"role": "user", "content": f"Real headline: {orig}"},
            {"role": "assistant", "content": sat}
        ]
    }
    fine_tuning_data.append(entry)

# Save in OpenAI JSONL format
fine_tuning_file = "fine_tuning_ready.jsonl"
with open(fine_tuning_file, "w") as f:
    for entry in fine_tuning_data:
        f.write(json.dumps(entry) + "\n")

print(f"Formatted dataset saved as {fine_tuning_file}")


Formatted dataset saved as fine_tuning_ready.jsonl


In [16]:
import openai

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Upload file for fine-tuning
with open("fine_tuning_ready.jsonl", "rb") as f:
    response = client.files.create(
        file=f,
        purpose="fine-tune"
    )

file_id = response.id  # Store file ID for fine-tuning step
print(f"File uploaded successfully! File ID: {file_id}")


File uploaded successfully! File ID: file-1knCTxQScERnhv4aKyqN1g


In [17]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo"  # GPT-3.5 is cheaper and faster than GPT-4
)

fine_tune_id = fine_tune_response.id
print(f"Fine-tuning job started! Job ID: {fine_tune_id}")


Fine-tuning job started! Job ID: ftjob-GNDMPv2TWpe1nO8uQkXQRJl5


In [24]:
job_status = client.fine_tuning.jobs.retrieve(fine_tune_id)
print(job_status)


FineTuningJob(id='ftjob-GNDMPv2TWpe1nO8uQkXQRJl5', created_at=1741773274, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:viddies::BADHydp0', finished_at=1741774060, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-f49DrDIit0rPl7avJxWBqmwR', result_files=['file-X19ohMeE3PswNyiknQ7E6z'], seed=670887314, status='succeeded', trained_tokens=34740, training_file='file-1knCTxQScERnhv4aKyqN1g', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None, metadata=None)


In [27]:
# List all fine-tuned models
models = client.models.list()

# Print out all fine-tuned models
for model in models:
    if "ft:gpt-3.5-turbo" in model.id:  # Filter for fine-tuned models
        print(f"Fine-Tuned Model ID: {model.id}")



Fine-Tuned Model ID: ft:gpt-3.5-turbo-0125:viddies::BADHyYA9:ckpt-step-100
Fine-Tuned Model ID: ft:gpt-3.5-turbo-0125:viddies::BADHyFHK:ckpt-step-200
Fine-Tuned Model ID: ft:gpt-3.5-turbo-0125:viddies::BADHydp0


In [28]:
fine_tuned_model_id = "ft:gpt-3.5-turbo-0125:viddies::BADHydp0"  # Replace with actual model ID

# Test with a sample real headline
test_headline = "Scientists discover a new planet with signs of water"

response = client.chat.completions.create(
    model=fine_tuned_model_id,
    messages=[
        {"role": "system", "content": "You are an AI that rewrites news headlines in a satirical, humorous style similar to The Onion."},
        {"role": "user", "content": f"Real headline: {test_headline}"}
    ]
)

# Print the generated satirical headline
print("Generated Satirical Headline:", response.choices[0].message.content)


Generated Satirical Headline: Astrophysicists Stumble Upon Previously Unknown Water Resort in the Galactic Suburbs.


In [29]:
# Compare vs GPT 3.5 turbo

response_base = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an AI that rewrites news headlines in a satirical, humorous style similar to The Onion."},
        {"role": "user", "content": f"Real headline: {test_headline}"}
    ]
)

print("Base GPT-3.5 Satirical Headline:", response_base.choices[0].message.content)


Base GPT-3.5 Satirical Headline: Scientists Shocked to Find Planet Not Entirely Covered in Dry Land


In [32]:
# Control Model Response at Runtime

# Temperature – Controls randomness. Higher (1.0) = more creative (hallucinations), Lower (0.2) = more predictable.
# Top-p (Nucleus Sampling) – Lower values (0.5) limit extreme randomness.
# Max Tokens – Limits response length, preventing long, unnecessary hallucinations.

response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:viddies::BADHydp0",
    messages=[
        {"role": "system", "content": "You are an AI that writes satirical headlines with a focus on realism."},
        {"role": "user", "content": "Real headline: Scientists discover a new planet with signs of water"}
    ],
    temperature=0.7,  # Higher for more creativity, lower for realism
    top_p=0.8,  # Reduce hallucination by limiting extreme randomness
    max_tokens=50  # Prevent overly long responses
)

print("Generated Satirical Headline:", response.choices[0].message.content)


Generated Satirical Headline: Scientists Uncover New Planet, Promptly Start Speculating on Whether It Could Support a Hipster Coffee Shop


In [35]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 122.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [37]:
import gradio as gr
import openai
import os

# Set fine-tuned model ID (Replace with your actual model ID)
fine_tuned_model_id = "ft:gpt-3.5-turbo-0125:viddies::BADHydp0"

# Function to generate satirical headlines
def generate_satirical_headline(real_headline, temperature=0.7):
    client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=[
            {"role": "system", "content": "You are an AI that rewrites news headlines in a satirical, humorous style similar to The Onion."},
            {"role": "user", "content": f"Real headline: {real_headline}"}
        ],
        temperature=temperature,  # Adjust randomness
        top_p=0.8,  # Control extreme hallucinations
        max_tokens=50  # Limit response length
    )

    return response.choices[0].message.content

# Create Gradio interface
iface = gr.Interface(
    fn=generate_satirical_headline,
    inputs=[
        gr.Textbox(label="Enter a Real News Headline"),
        gr.Slider(0.2, 1.2, value=0.7, step=0.1, label="Creativity (Temperature)")
    ],
    outputs="text",
    title="📰 Satirical Headline Generator",
    description="Enter a real news headline and get a satirical version inspired by The Onion!"
)

# Launch Gradio app
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45c4e44cce75cfd60c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
